In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
# CHAT

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!")
]

model = ChatOpenAI(model="gpt-4o")
result = model.invoke(messages)
StrOutputParser().invoke(result) # or result.content

'Ciao!'

In [14]:
# CHAIN

model = ChatOpenAI(model="gpt-4o")
parser = StrOutputParser()

chain = model | parser
chain.invoke(messages)

'Ciao!'

In [16]:
# PROMPT TEMPLATE (parameterize prompt)

from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "{text}")
])

# invoke the prompt template to see how it works by itself
prompt_result = prompt_template.invoke({ "language": "italian", "text": "hi" })
print(prompt_result.to_messages())

[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')]


In [18]:
chain = prompt_template | model | parser
chain.invoke({ "language": "italian", "text": "hi" })

'Ciao'

In [23]:
# CHAT MEMORY (conversational state)

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hello, Bob! How can I assist you today?'